Importación de librerías

In [ ]:
import pandas as pd
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

In [16]:
#data = pd.read_csv('lp5.csv', sep=',')
data = np.loadtxt('lp5.csv', delimiter=',',dtype=str)
data

array([['normal', '', '', '', '', ''],
       ['-2', '-1', '81', '0', '-5', '0'],
       ['-2', '-1', '79', '0', '-4', '0'],
       ...,
       ['-1', '1', '-3', '-3', '-2', '-3'],
       ['0', '-1', '-5', '-1', '1', '-3'],
       ['-1', '1', '4', '0', '-1', '-3']], dtype='<U18')

Separo las categorías y las matrices en listas por aparte

In [28]:
count = 0
matriz = []
label = []


for i in range(2624): 
    if (count==0):
        label.append((data[i]))
    if (count > 0):
        matriz.append((data[i]))
    count+=1 
    if (count == 16): 
      count = 0
print(len(label))
print(len(matriz))

164
2460


Añado los datos de la lista "matriz" a una matriz

In [33]:
matriz_data=np.zeros((len(matriz), 6))

for i in range(2460):
    for j in range(6):
        matriz_data[i][j] = matriz[i][j]

array([[-2., -1., 81.,  0., -5.,  0.],
       [-2., -1., 79.,  0., -4.,  0.],
       [-2., -1., 79.,  0., -4.,  0.],
       ...,
       [-1.,  1., -3., -3., -2., -3.],
       [ 0., -1., -5., -1.,  1., -3.],
       [-1.,  1.,  4.,  0., -1., -3.]])

Uno la matriz y los labels en una sola matriz

In [ ]:
count = 0
mat_final=np.zeros((164,15,6))
for i in range(164): 
    for j in range(15):
        for k in range(6):
            mat_final[i][j][k] = matriz[count][k] 
        count += 1 